#### Testowe dane

In [3]:
import numpy as np
X_train=np.array([[10,11], [20,30], [5,4], [8,20], [22, 10], [9, 10], [5, 17], [5, 50], [50,5], [3,4], [6,7], [-10,10], [-10,1]])
y_train=np.array([0,1, 0, 1, 0, 0, 1,1,0,0,0, 1,1])
assert len(X_train)==len(y_train)


X_test=np.array([[0,10], [20,30], [-20, 1], [19,20], [10,5], [2,1]])
y_test=np.array([1,1,1,0,0,0])
assert len(X_test)==len(y_test)

#### Implementacja perceptronu

In [221]:
class Perceptron:
    
    def __init__(self, num_features, num_epoch = 1000, learning_rate = 0.01):
        self.num_features = num_features
        self.num_epoch = num_epoch
        self.learning_rate = learning_rate
        self.weights = np.random.normal(0, 1, self.num_features + 1)
    
    @staticmethod
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
    
    def __forward(self, x):
        y_prob = self.sigmoid(np.dot(x, self.weights[1:]) + self.weights[0])
        return y_prob
    
    def train(self, X, Y, X_test, Y_test):
        
        for epoch in range(self.num_epoch):
            loss = 0
            y_true = 0
            
            for x, y in zip(X, Y):
                y_prob = self.__forward(x)
                y_pred = 1 if y_prob > 0.5 else 0
                if y_pred == y:
                    y_true += 1
                loss += 0.5 * (y - y_prob) ** 2
                self.weights[1:] += self.learning_rate * (y - y_prob) * y_prob * (1 - y_prob) * x
                self.weights[0] += self.learning_rate * (y - y_prob) * y_prob * (1 - y_prob)
                
            pred = self.predict(X_test)
            acc = sum([ y == y_ for y, y_ in zip(pred, Y_test)]) / Y_test.shape[0]
            
            print('Epoch:', str(epoch + 1), '/', str(self.num_epoch), '|', 'loss:', loss, '|',  'train accuracy:', round(y_true / X.shape[0] * 100, 2), '%', '|', 'test accuracy:', round(acc * 100, 2), '%.')
           
    def predict(self, X):
        y_prob = self.__forward(X)
        vfunc = np.vectorize(lambda x: 1 if x > 0.5 else 0)
        return vfunc(y_prob)

#### Trenowanie modelu

In [226]:
model = Perceptron(2)
model.train(X_train, y_train, X_test, y_test)

Epoch: 1 / 1000 | loss: 2.363118186125232 | train accuracy: 61.54 % | test accuracy: 66.67 %.
Epoch: 2 / 1000 | loss: 2.2936886741316957 | train accuracy: 61.54 % | test accuracy: 66.67 %.
Epoch: 3 / 1000 | loss: 2.2292095182952765 | train accuracy: 61.54 % | test accuracy: 66.67 %.
Epoch: 4 / 1000 | loss: 2.1753585508745323 | train accuracy: 69.23 % | test accuracy: 66.67 %.
Epoch: 5 / 1000 | loss: 2.1344719858962464 | train accuracy: 69.23 % | test accuracy: 66.67 %.
...


Epoch: 995 / 1000 | loss: 0.0556951382895211 | train accuracy: 100.0 % | test accuracy: 100.0 %.
Epoch: 996 / 1000 | loss: 0.05564351589891698 | train accuracy: 100.0 % | test accuracy: 100.0 %.
Epoch: 997 / 1000 | loss: 0.05559198365795457 | train accuracy: 100.0 % | test accuracy: 100.0 %.
Epoch: 998 / 1000 | loss: 0.05554054133036015 | train accuracy: 100.0 % | test accuracy: 100.0 %.
Epoch: 999 / 1000 | loss: 0.05548918868070455 | train accuracy: 100.0 % | test accuracy: 100.0 %.
Epoch: 1000 / 1000 | loss: 0.05543792547439876 | train accuracy: 100.0 % | test accuracy: 100.0 %.


#### Perceptron w PyTorch

In [300]:
import torch.nn as nn
import torch

class PerceptronTorch(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)     # jedna warstwa modelu - liniowa
        self.sigm = nn.Sigmoid()                           # funkcja aktywacji sigmoidalna

    def forward(self, x): 
        out = self.linear(x)                               # obliczanie wyjścia
        out = self.sigm(out)                               # przeksztalcenie sigmoidalne
        return out

In [311]:
def train_perceptron_torch():
    
    # Params
    torch.manual_seed(717)
    epochs = 1000
    lr_rate = 0.01
    
    # Model
    model = PerceptronTorch(input_dim = 2, output_dim = 1)
    
    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr = lr_rate)       
    
    # Loss
    criterion = nn.MSELoss()                                             # funkcja mse jest analogiczna do funkcji straty
                                                                         # z modelu pierwszego tylko nie przemnozona przez 0.5 co nie ma wplywu na gradient
    # Train data
    inputs = torch.from_numpy(X_train).view(-1,2).float()                # konwersja do float
    labels = torch.from_numpy(y_train).float()                           # konwersja do float
    
    # Test data
    inputs_test = torch.from_numpy(X_test).view(-1,2).float()            # konwersja do float
    
    # Train
    for epoch in range(1, epochs + 1, 1):
        
        optimizer.zero_grad()                                            # wyczyszczenie gradientu
        outputs = model(inputs)                                          # przeksztalcenie danych
        
        loss = criterion(outputs, labels.unsqueeze(1))                   # obliczenie loss
        loss.backward()                                                  
        
        optimizer.step()                                                 # aktualizacja wag
        
        # Eval train
        y_prob = model(inputs).detach().numpy()                          # predykcja
        y_pred = [1 if y_ > 0.5 else 0 for y_ in y_prob]                 # obliczenie labels gdzie prog ustalam na 0.5
        acc_train = sum([ y == y_ for y, y_ in zip(y_pred, y_train)]) / y_train.shape[0]
        
        # Evala test
        y_prob_test = model(inputs_test).detach().numpy()                # predykcja
        y_pred_test = [1 if y_ > 0.5 else 0 for y_ in y_prob_test]       # obliczenie labels gdzie prog ustalam na 0.5
        acc_test= sum([ y == y_ for y, y_ in zip(y_pred_test, y_test)]) / y_test.shape[0]
        
        # Print results
        print('epoch {} | loss {} | accuracy train {}% | accuracy test {}%'.format(epoch, loss.item(), round(acc_train * 100, 2), round(acc_test * 100, 2)))

In [312]:
train_perceptron_torch()

epoch 1 | loss 0.643459677696228 | accuracy train 30.77% | accuracy test 33.33%
epoch 2 | loss 0.6429001688957214 | accuracy train 30.77% | accuracy test 33.33%
epoch 3 | loss 0.6423274278640747 | accuracy train 30.77% | accuracy test 33.33%
epoch 4 | loss 0.6417413353919983 | accuracy train 30.77% | accuracy test 33.33%
...


epoch 995 | loss 0.020326267927885056 | accuracy train 100.0% | accuracy test 100.0%
epoch 996 | loss 0.020314179360866547 | accuracy train 100.0% | accuracy test 100.0%
epoch 997 | loss 0.02030211314558983 | accuracy train 100.0% | accuracy test 100.0%
epoch 998 | loss 0.020290056243538857 | accuracy train 100.0% | accuracy test 100.0%
epoch 999 | loss 0.020278016105294228 | accuracy train 100.0% | accuracy test 100.0%
epoch 1000 | loss 0.020266003906726837 | accuracy train 100.0% | accuracy test 100.0%
